Была взята статья Доксы о фильмах. Трудные для машинной разметки места: много неологизмов, составных слов, названий и имен.

Выбранный тегсет: https://universaldependencies.org/u/pos/index.html - есть подробное описание, довольно сжатый, поэтому с ним легче сравнивать более подробные теги. 


In [1]:
import re
import pandas as pd
from tqdm.auto import tqdm


In [2]:
with open('text.txt', 'r', encoding='utf-8') as text_file:
    text = text_file.read()
    

In [3]:
text = re.sub('[^\w\d\s\-]', '', text)
text = re.sub(' +', ' ', text)
text


'Фильм родился благодаря большому проекту 100 лет настоящего который проходил в Доме мировых культур в Берлине с 2015 по 2019 год Он состоял из многочисленных творческих и образовательных мероприятий и значительную роль в нем играли лекции самых разных гуманитарных исследователей Работа Михаэля Буша его личная интерпретация этого проекта в формате экспериментального видеоколлажа Главная идея которой руководствовался режиссер при его создании связана с Тезисами о философии истории Вальтера Беньямина и в частности с его специфическим понимании времени Фильм задал тон структуре всей программы и подарил ей своей название Ойкономия изучает природу современного финансового капитализма и пытается описать столпы на которых он держится например идею экономического роста Зрителю предлагается задуматься в какой момент и почему наша экономика может существовать только в режиме роста а также какую роль в ней играет долг Фильм позволяет пристально рассмотреть ключевой элемент базиса нашего общеcтвен

In [4]:
def check_pos(word):
    mstm = get_mystem_pos(word)
    slvnt = get_slovnet_pos(word)
    pmrph = get_pymorphy_pos(word)
    return [mstm, slvnt, pmrph]


In [5]:
from pymystem3 import Mystem

def get_mystem_pos(word):
    m = Mystem()
    mystem_pos = []
    mystem_word = []
    words = m.analyze(word)
    ans = []
    for word in words:
        try:
            pos = re.findall('[A-Z]+', word['analysis'][0]['gr'])[0]
            ans.append(pos)
        except:
            if len(re.findall('\d+', word['text'])) > 0:
                ans.append('None')
            pass
    if len(ans) == 1:
        return str(ans).rstrip("']").lstrip("['")
    else:
        return re.sub("'",'', str(ans).rstrip(']').lstrip('['))
    

In [6]:
from razdel import sentenize, tokenize
from navec import Navec
from slovnet import Morph

def get_slovnet_pos(word):
    chunk = []
    for sent in sentenize(word):
        tokens = [_.text for _ in tokenize(sent.text)]
        chunk.append(tokens)

    navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
    morph = Morph.load('slovnet_morph_news_v1.tar', batch_size=4)
    morph.navec(navec)

    markup = next(morph.map(chunk))
    ans = []
    for token in markup.tokens:
        ans.append(token.pos)
    if len(ans) == 1:
        return str(ans).rstrip("']").lstrip("['")
    else:
        return re.sub("'",'', str(ans).rstrip(']').lstrip('['))
    

In [7]:
import pymorphy2

def get_pymorphy_pos(word):
    morph = pymorphy2.MorphAnalyzer()
    tokenized_text = word.split(' ')
    ans = []
    for word in tokenized_text:
        ans.append(morph.parse(word)[0].tag.POS)
    if len(ans) == 1:
        return str(ans).rstrip("']").lstrip("['")
    else:
        return re.sub("'",'', str(ans).rstrip(']').lstrip('['))
        

In [8]:
text_words = text.split(' ')
mystem_poses = []
slovnet_poses = []
pymorphy_poses = []
for word in tqdm(range(len(text_words))):
    get_all = check_pos(text_words[word])
    mystem_poses.append(get_all[0])
    slovnet_poses.append(get_all[1])
    pymorphy_poses.append(get_all[2])
    

In [9]:
with open('corp.txt', 'r', encoding='utf-8') as my_corp:
    corp = my_corp.read().split('\n')
    my_poses = []
    for word in corp:
        pos = word.split('\t')[1]
        my_poses.append(pos)


In [10]:
df = pd.DataFrame(list(zip(text_words, my_poses, slovnet_poses, mystem_poses, pymorphy_poses)), columns =['word', 'my_pos', 'slovnet_pos', 'mystem_pos', 'pymorphy_pos'])

df


,word,my_pos,slovnet_pos,mystem_pos,pymorphy_pos
0,Фильм,NOUN,NOUN,S,NOUN
1,родился,VERB,VERB,V,VERB
2,благодаря,ADP,ADP,PR,PREP
3,большому,ADJ,ADJ,A,ADJF
4,проекту,NOUN,NOUN,S,NOUN
...,...,...,...,...,...
340,вопросы,NOUN,NOUN,S,NOUN
341,кажутся,VERB,ADJ,V,VERB
342,мне,PRON,PRON,SPRO,NPRO
343,невероятно,ADV,ADV,ADV,ADVB


# Считаем accuracy

In [11]:
pymorphy_acc = {
    'ADJ': ['ADJF', 'ADJS', 'COMP'],
    'ADP': ['PREP'],
    'ADV': ['ADVB', 'PRED'],
    'CCONJ': ['CONJ'],
    'DET': ['NOUN'],
    'NOUN': ['NOUN'],
    'NUM': ['NUMR'],
    'PART': ['PRCL'],
    'PRON': ['NPRO'],
    'PROPN': ['NOUN'],
    'SCONJ': ['CONJ'],
    'VERB': ['VERB',  'INFN', 'PRTF', 'PRTS', 'GRND']
}


In [12]:
mystem_acc = {
    'ADJ': ['A', 'APRO'],
    'ADP': ['PR'],
    'ADV': ['ADV', 'ADVPRO'],
    'CCONJ': ['CONJ'],
    'DET': ['APRO', 'ADVPRO'],
    'NOUN': ['S', 'S, S'],
    'NUM': ['NUM'],
    'PART': ['PART'],
    'PRON': ['SPRO'],
    'PROPN': ['S'],
    'SCONJ': ['CONJ'],
    'VERB': ['V']
}


In [13]:
def check_accuracy(acc, machine_tags, my_tags):
    acc_counter = 0
    for i in range(len(machine_tags)):
        if machine_tags[i] in acc[my_tags[i]]:
            acc_counter += 1
    return acc_counter/len(machine_tags)


In [14]:
counter = 0
for i in range(len(slovnet_poses)):
    if slovnet_poses[i] == my_poses[i]:
        counter += 1
        
slovnet_accuracy = counter/len(slovnet_poses)
mystem_accuracy = check_accuracy(mystem_acc, mystem_poses, my_poses)
pymorphy_accuracy = check_accuracy(pymorphy_acc, pymorphy_poses, my_poses)


In [15]:
slovnet_accuracy


0.9130434782608695

In [16]:
mystem_accuracy


0.855072463768116

In [17]:
pymorphy_accuracy


0.881159420289855

# Находим три типа n-грамм

In [18]:
def get_slovnet_ngram(text, ngram_type):
    chunk = []
    for sent in sentenize(text):
        tokens = [_.text for _ in tokenize(sent.text)]
        chunk.append(tokens)

    navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
    morph = Morph.load('slovnet_morph_news_v1.tar', batch_size=4)
    morph.navec(navec)
    
    markup = next(morph.map(chunk))
    ngram_tags = ''
    for token in markup.tokens:
        ngram_tags += token.pos
        ngram_tags += ','
    if ngram_tags.rstrip(',') == ngram_type:
        return text
    

In [19]:
ngram_types = ['NOUN,VERB', 'ADJ,NOUN', 'ADV,VERB']
for ngram_type in ngram_types:
    ngram_parts = ngram_type.split(',')
    n = len(ngram_parts)
    grams = [text_words[i: i + n] for i in range(len(text_words)- n + 1)]
    ans = []
    for gram in grams:
        if get_slovnet_ngram(' '.join(gram), ngram_type):
            ans.append(' '.join(gram))
    print(ngram_type, ans)
    

NOUN,VERB ['Фильм родился', 'создании связана', 'Фильм задал', 'Зрителю предлагается', 'экономика может', 'Фильм позволяет', 'вопросы связанные', 'кадре появляются', 'дронов демонстрируя', 'Фильм описывает', 'контроля рисуя', 'коллаж состоящий', 'сторон рассматривают', 'Режиссер исследует', 'установки определяют', 'желания формируют', 'процессах играют']
ADJ,NOUN ['большому проекту', 'мировых культур', '2019 год', 'образовательных мероприятий', 'значительную роль', 'гуманитарных исследователей', 'личная интерпретация', 'экспериментального видеоколлажа', 'Главная идея', 'специфическим понимании', 'финансового капитализма', 'экономического роста', 'ключевой элемент', 'порочную логику', 'глобального мира', 'разного профиля', 'экологической обстановкой', 'технологических систем', 'современных дронов', 'визуальной культуры', 'тотального контроля', 'тревожный антиутопический', 'антиутопический горизонт', 'сама картина', 'меньшей степени', 'разнородных эпизодов', 'разных техниках', 'резонансн